# Тема 3. Дифференцирование таблично заданной функции с помощью многочлена Лагранжа

In [118]:
# %run LR1.ipynb

In [119]:
from pprint import pprint

import numpy as np
import math
import pandas as pd
import sympy as sp

In [120]:
# Исходные данные (из ЛР №1)
x_sym = sp.Symbol('x')
h_sym = sp.Symbol('h')
f_sym = (x_sym-1)**2 - 0.5*sp.exp(x_sym)

a, b = 0.1, 0.6
N = 10
h = (b - a) / N

In [121]:
# Параметры варианта
k = 1  # порядок производной
n = 5  # степень многочлена
m = 0  # индекс точки

In [122]:
# Таблица узлов и значений
x_nodes = np.linspace(a, b, N + 1)
x_nodes_sym = sp.symbols(f'x_0:{n+1}')
x_nodes_sym_value = {sym: val for sym, val in zip(x_nodes_sym, x_nodes)}

f_numeric = sp.lambdify(x_sym, f_sym, 'numpy')
y_nodes = f_numeric(x_nodes)
y_nodes_sym = sp.symbols(f'f_0:{n+1}')
y_nodes_sym_value = {sym: val for sym, val in zip(y_nodes_sym, y_nodes)}

In [123]:
# Выбор узлов
x_m = x_nodes[m]
pprint(x_nodes)
pprint(x_m)

array([0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55, 0.6 ])
np.float64(0.1)


In [124]:
# Подготовка оригинальной формулы
f_deriv_sym = sp.diff(f_sym, x_sym, k)

In [125]:
# Создание формулы Лагранжа
def lagrange_basis(i):
    """Символьный базисный полином Лагранжа"""
    result = 1
    for j in range(n):
        if j != i:
            result *= (x_sym - x_nodes[j]) / ((i - j) * h_sym)
            # result *= (x_sym - x_nodes[j]) / (x_nodes[i] - x_nodes[j])
    return result

def lagrange_polynomial():
    """Символьный интерполяционный многочлен Лагранжа"""
    L_n = 0
    for i in range(n):
        basis = lagrange_basis(i)
        L_n += y_nodes_sym[i] * basis
    return sp.simplify(L_n)

L_n_sym = lagrange_polynomial()

In [126]:
# Подготовка формулы Лагранжа
L_n_deriv_sym = sp.diff(L_n_sym, x_sym, k)
L_n_deriv_sym = sp.simplify(L_n_deriv_sym)
L_n_deriv_sym

0.0416666666666667*(4.0*f_0*x**3 - 2.7*f_0*x**2 + 0.595*f_0*x - 0.04275*f_0 - 16.0*f_1*x**3 + 10.2*f_1*x**2 - 2.08*f_1*x + 0.134*f_1 + 24.0*f_2*x**3 - 14.4*f_2*x**2 + 2.73*f_2*x - 0.162*f_2 - 16.0*f_3*x**3 + 9.0*f_3*x**2 - 1.6*f_3*x + 0.09*f_3 + 4.0*f_4*x**3 - 2.1*f_4*x**2 + 0.355*f_4*x - 0.01925*f_4)/h**4

In [127]:
# Расчёт оригинальной формулы
f_deriv_sym = f_deriv_sym.subs({"x": x_m})
f_deriv_sym

-2.35258545903782

In [128]:
# Расчёт формулы Лагранжа
L_n_deriv_sym = L_n_deriv_sym.subs({"x": x_m})
L_n_deriv_sym = L_n_deriv_sym.subs({"h": h})
# L_n_deriv_sym = L_n_deriv_sym.subs(x_nodes_sym_value)
L_n_deriv_sym = L_n_deriv_sym.subs(y_nodes_sym_value)
L_n_deriv_sym

-2.35258470798014

In [129]:
print("результаты")
print(f"Точка x_{m} = {x_m:.6f}")
print(f"f^({k})(x_{m}) = {f_deriv_sym:.10f}")
print(f"L_{n}^({k})(x_{m}) = {L_n_deriv_sym:.10f}")
real_R_nk = f_deriv_sym - L_n_deriv_sym
print(f"Погрешность R_{n},{k} = {real_R_nk:.10f}")
print(f"Погрешность R_{n},{k} = {real_R_nk:.6e}")

результаты
Точка x_0 = 0.100000
f^(1)(x_0) = -2.3525854590
L_5^(1)(x_0) = -2.3525847080
Погрешность R_5,1 = -0.0000007511
Погрешность R_5,1 = -7.510577e-7


In [130]:
# остаточный член
def get_omega():
    """Символьная функция ω_{n+1}(x) = (x - x_0)(x - x_1)...(x - x_n)"""
    result = 1
    for i in range(n):
        result *= (x_sym - x_nodes[i])
    return sp.simplify(result)

def get_R_nk(f_deriv_n1=None):
    omega_n1 = get_omega()
    omega_deriv = sp.diff(omega_n1, x_sym, k)
    omega_deriv = sp.simplify(omega_deriv)

    if f_deriv_n1 is None:
        f_deriv_n1 = sp.diff(f_sym, x_sym, n + 1)

    R_nk = (f_deriv_n1 / sp.factorial(n + 1)) * omega_deriv
    return sp.simplify(R_nk)

R_nk_sym = get_R_nk()
R_nk_sym


(-0.00347222222222222*x**4 + 0.00277777777777778*x**3 - 0.000807291666666667*x**2 + 0.000100694444444444*x - 4.53125e-6)*exp(x)

In [131]:
def get_min_max_of_R_nk():
    f_deriv_n1 = sp.diff(f_sym, x_sym, n + 1)
    f_deriv_n1_prime = sp.diff(f_deriv_n1, x_sym)
    critical_points = sp.solve(f_deriv_n1_prime, x_sym)
    critical_points = [sp.re(i).evalf() for i in critical_points]
    critical_points = [i for i in critical_points if a <= i <= b]
    critical_points.append(a)
    critical_points.append(b)
    values = [f_deriv_n1_prime.subs({"x": point}) for point in critical_points]
    min_f_deriv_n1, max_f_deriv_n1 = min(values), max(values)
    return get_R_nk(min_f_deriv_n1), get_R_nk(max_f_deriv_n1)


R_nk_min, R_nk_max = get_min_max_of_R_nk()
R_nk_min, R_nk_max

(-0.00632680139024482*x**4 + 0.00506144111219586*x**3 - 0.00147098132323192*x**2 + 0.0001834772403171*x - 8.2564758142695e-6,
 -0.003837399021096*x**4 + 0.0030699192168768*x**3 - 0.00089219527240482*x**2 + 0.000111284571611784*x - 5.00780572253028e-6)

In [134]:
R_nk_min_val = R_nk_min.subs({"x": x_m})
R_nk_max_val = R_nk_max.subs({"x": x_m})
R_nk_min_val, R_nk_max_val

(-1.89804041707345e-7, -1.15121970632881e-7)

In [133]:
print("результаты")
print(f"Точка x_{m} = {x_m:.6f}")
print(f"f^({k})(x_{m}) = {f_deriv_sym:.10f}")
print(f"L_{n}^({k})(x_{m}) = {L_n_deriv_sym:.10f}")
real_R_nk = f_deriv_sym - L_n_deriv_sym
print(f"Погрешность R_{n},{k} = {real_R_nk:.10f}")
print(f"Погрешность R_{n},{k} = {real_R_nk:.6e}")

print(f"Оценка погрешности с min(R_{n},{k}) = {(real_R_nk - R_nk_min_val):.10f}")
print(f"Оценка погрешности с min(R_{n},{k}) = {(real_R_nk - R_nk_min_val):.6e}")

print(f"Оценка погрешности с max(R_{n},{k}) = {(R_nk_max_val - real_R_nk):.10f}")
print(f"Оценка погрешности с max(R_{n},{k}) = {(R_nk_max_val - real_R_nk):.6e}")

результаты
Точка x_0 = 0.100000
f^(1)(x_0) = -2.3525854590
L_5^(1)(x_0) = -2.3525847080
Погрешность R_5,1 = -0.0000007511
Погрешность R_5,1 = -7.510577e-7
Оценка погрешности с min(R_5,1) = -0.0000005613
Оценка погрешности с min(R_5,1) = -5.612536e-7
Оценка погрешности с max(R_5,1) = 0.0000006359
Оценка погрешности с max(R_5,1) = 6.359357e-7
